In [137]:
# !pip install pyomo
# !pyomo install-extras
#!conda install -y -c conda-forge pyomo.extras
#!conda install -y -c conda-forge glpk
import random
import numpy as np
import pandas as pd
import pdb
import math
%matplotlib inline 

In [1537]:
from pyomo.environ import *
import pyomo.environ as pe 

A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14
model = ConcreteModel()
model.x = Var( A, within=Binary )
model.value = Objective(expr = sum( b[i]*model.x[i] for i in A), sense = maximize )
model.weight = Constraint(expr = sum( w[i]*model.x[i] for i in A) <= W_max )
opt = SolverFactory('glpk')
result_obj = opt.solve(model)
model.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['hammer', 'screwdriver', 'towel', 'wrench']

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf : 5*x[hammer] + 7*x[wrench] + 4*x[screwdriver] + 3*x[t

In [3]:
model.x.get_values()

{'screwdriver': 1.0, 'towel': 1.0, 'wrench': 0.0, 'hammer': 1.0}

### Optimize

In [1534]:
def generate_schedual(clients):
    schedual = []
    for client in clients:
        num_slots = random.randint(1, 2)
        for i in range(num_slots):
            start = float(random.randint(8, 14)) + random.randint(0, 4) * 0.25
            duration = 0.25 * random.randint(1, 4)
            end = start + duration
            weight = random.randint(1, 5)
            schedual += [[client, start, end, duration, weight]]
    return schedual

In [4]:
from collections import defaultdict
tasks = [["T1", 08.50, 09.50, 1.00, 3], 
         ["T2", 09.25, 10.00, 0.75, 4],
         ["T3", 09.50, 10.75, 0.50, 2],
         ["T4", 10.25, 11.50, 0.25, 1],
         ["T5", 12.00, 13.50, 0.75, 1],
         ["T6", 12.25, 13.75, 0.75, 2],
         ["T1", 12.00, 13.50, 1.00, 3]]

def find_possible_slots(start_time, end_time, duration):
    for i in range(int((end_time - start_time) / 0.25)):
        end_time_ = start_time + i * 0.25 + duration
        if end_time_ <= end_time:
            yield (start_time + i * 0.25, start_time + i * 0.25 + duration, duration)

def task_id(task, slot):
    return {"task": task[0] + "_" + str(slot[0]) + "_" + str(slot[1]), 
            "task_group": task[0], 
            "start": slot[0], 
            "finish": slot[1], 
            "duration": slot[2], 
            "weight": task[4]}

def compute_possible_tasks(tasks):
    tasks_ = []
    for task in tasks:
        slots = find_possible_slots(float(task[1]), float(task[2]), float(task[3]))
        for slot in slots:
            tasks_ += [task_id(task, slot)]
    tasks_  = sorted(tasks_, key=lambda x: x["finish"])
    return pd.DataFrame(tasks_)




In [5]:
def find_task_overlaps(possible_tasks):
    overlapping_tasks = {}
    for task in possible_tasks.task.unique():
        start, finish = possible_tasks[possible_tasks.task == task].iloc[0][["start", "finish"]].values
        overlapping_tasks_ = possible_tasks[(possible_tasks.start >= start) & (possible_tasks.start < finish)]
        overlapping_tasks_ = list(overlapping_tasks_.task.unique())
        if len(overlapping_tasks_) > 1:
            overlapping_tasks[task] = overlapping_tasks_
    return overlapping_tasks

def find_task_groups(possible_tasks):
    return possible_tasks["task_group"].unique()

In [1538]:
def schedual_tasks(tasks):
    model = ConcreteModel()

    #data
    possible_tasks = compute_possible_tasks(tasks)
    tasks = possible_tasks.task.values
    w = {r.task: r.weight for _, r in possible_tasks[possible_tasks.task == tasks].iterrows()}

    #constarint data
    overlapping_tasks = find_task_overlaps(possible_tasks)
    task_groups = find_task_groups(possible_tasks)

    #variables
    model.x = Var(tasks, within=Binary )

    #objective
    model.value = Objective(expr = sum(w[i]*model.x[i] for i in tasks), sense = maximize )

    #constraints
    @model.Constraint(task_groups)
    def one_each_group(m, tg):
        return sum(m.x[task] for task in possible_tasks[possible_tasks["task_group"] == tg]["task"].unique()) <= 1

    @model.Constraint(overlapping_tasks.keys())
    def one_each_overlap(m, t):
        return sum(m.x[task] for task in overlapping_tasks[t]) <= 1

    #solve
    opt = SolverFactory('glpk')
    result_obj = opt.solve(model)
    selected = [k for k, v in model.x.get_values().items() if v == 1]
    
    #formate resutls
    results = (possible_tasks
          .loc[possible_tasks.task.isin(selected)]
          .sort_values(by=['start'])
          .set_index("task_group")
          [["start", "finish", "duration", "weight"]])
    return results


In [1539]:
%%capture
results = schedual_tasks(tasks)


In [1540]:
results

,start,finish,duration,weight
task_group,,,,
T2,9.25,10.00,0.75,4
T3,10.00,10.50,0.50,2
T4,10.50,10.75,0.25,1
T1,12.00,13.00,1.00,3
T6,13.00,13.75,0.75,2


In [1541]:
results.weight.sum()

12

In [1542]:
def solve_by_elemination(tasks):
    schedual = []
    possible_tasks = compute_possible_tasks(tasks)
    possible_tasks_ = possible_tasks.copy().sort_values(by=['weight'], ascending=False)
    for i in range(100):
        try:
            task, task_group, start, finish = possible_tasks_.iloc[i][["task", "task_group", "start", "finish"]]
            possible_tasks_ = possible_tasks_[~((possible_tasks_.start >= start) & (possible_tasks_.start < finish) & ((possible_tasks_.task != task)))]
            possible_tasks_ = possible_tasks_[~((possible_tasks_.finish > start) & (possible_tasks_.finish <= finish) & ((possible_tasks_.task != task)))]
            possible_tasks_ = possible_tasks_[(possible_tasks_.task_group != task_group) | (possible_tasks_.task == task ) ]
            schedual += [task]
        except:
            break
    return possible_tasks[possible_tasks.task.isin(schedual)]

solve_by_elemination(tasks).weight.sum()

10

### We don't know the weights

In [1543]:
class Oracle:
    def observe(self, problem, solution):
        pass
        
class Solver:
    def solve(self, problem, weights):
        pass
    
class Problem:
    pass

class ProblemModel:
    def get_all_weights(self):
        pass
    
    def set_weights(self, weights):
        pass
    


In [1544]:
class TaskSchedualingProblem(Problem):
    def __init__(self, tasks):
        self.tasks = tasks


class TaskSchedualingProblemModel(ProblemModel):
    def __init__(self, tasks, weights_known=False):
        if weights_known:
            self.weights = dict(compute_possible_tasks(tasks)[["task_group", "weight"]].drop_duplicates().values)
        else:
            self.weights = {i: random.random() for i in compute_possible_tasks(tasks)["task_group"].drop_duplicates().values}
        self.arms = self.weights.keys()  
        
    def get_all_weights(self):
        return self.weights
    
    def set_weights(self, weights):
        for k, v in weights.items():
            self.weights[k] = v
    

class TaskSchedualingSolver(Solver):
    def __init__(self):
        pass 
    
    def solve(self, problem, weights):
        tasks = [task[:4] + [weights[task[0]]] for task in problem.tasks]
        return schedual_tasks(tasks).index.values
        
        
class TaskSchedualingOracle(Oracle):
    def __init__(self, tasks, noise_factor=3.0):
        self.weights = dict(compute_possible_tasks(tasks)[["task_group", "weight"]].drop_duplicates().values)
        self.arms = self.weights.keys()  
        self.noise_factor = noise_factor
        
    def get_weight(self, x, noisy=False):
        if noisy:
            return self.weights[x] + (random.random() - 0.5) * self.noise_factor
        else:
            return self.weights[x]
        
    def observe(self, problem, solution, noisy=False):
        return [self.get_weight(x, noisy=noisy) for x in solution]
        
        
class CombUcb1:
    def __init__(self, problem, problem_model, solver, oracle, mode='Max'):
        self.problem = problem
        self.solver = solver
        self.oracle = oracle
        self.arms = problem_model.arms
        self.mode = mode
        self.init_algorithm()

    def init_algorithm(self):
        uu = {i: 0.0 for i in self.arms}
        w = {i: 0.0 for i in self.arms}
        t = 0
        for arm in uu.keys():
            if self.mode == 'Max':
                uu[arm] = 1.0
            elif self.mode == 'Min':
                uu[arm] = 0.0
            else:
                raise Exception('Mode is only Max or Min')
        solution_exists = True

        while ((self.mode == 'Min' and np.min(list(uu.values())) == 0)
                                        or (self.mode == 'Max' and np.max(list(uu.values())) == 1.0)):
            At = self.solver.solve(self.problem, uu)
            if At is None:
                break
            AtW = self.oracle.observe(self.problem, At, noisy=True)

            for idx, e in enumerate(At):
                w[e] = AtW[idx]
                if self.mode == 'Max':
                    uu[e] = 0.0
                elif self.mode == 'Min':
                    uu[e] = 1.0
                else:
                    raise Exception('Mode is only Max or Min')
            t += 1
        self.weights = w
        self.time_steps = {i: 1.0 for i in w.keys()}
        self.t = t

    def bandit_iter(self):
        weights = self.weights
        time_steps = self.time_steps
        t = self.t
        if self.mode == 'Max':
            u_ucb = {i: min(weights[i] + np.sqrt(1.5 * np.log(t)/time_steps[i]), 1.0) for i in weights.keys()}
        elif self.mode == 'Min':
            u_ucb = {i: max(weights[i] - np.sqrt(1.5 * np.log(t)/time_steps[i]), 0) for i in weights.keys()}
        else:
            raise Exception('Mode is only Max or Min')
        At = self.solver.solve(self.problem, u_ucb)
        wAt = self.oracle.observe(self.problem, At)
        for idx, e in enumerate(At):
            weights[e] = (time_steps[e] * weights[e] + wAt[idx]) / (time_steps[e] + 1)
            time_steps[e] += 1
        t += 1
        self.time_steps = time_steps
        self.weights = weights
        self.t = t

    def solve(self):
        weights = self.weights
        u_ucb = weights
        return self.solver.solve(self.problem, u_ucb)


In [1545]:
problem = TaskSchedualingProblem(tasks)
problem_model = TaskSchedualingProblemModel(tasks, weights_known=True)
oracle = TaskSchedualingOracle(tasks, noise_factor=5)
solver = TaskSchedualingSolver()
sln = solver.solve(problem, problem_model.get_all_weights())
sum(oracle.observe(problem, sln, noisy=False))

12

In [1546]:
problem_model_uncertain = TaskSchedualingProblemModel(tasks, weights_known=False)
sln = solver.solve(problem, problem_model_uncertain.get_all_weights())
sum(oracle.observe(problem, sln, noisy=False))


10

In [1547]:
%%capture 
b = CombUcb1(problem=problem, problem_model=problem_model_uncertain, solver=solver, oracle=oracle, mode='Max')

for i in range(5):
    print (i)
    b.bandit_iter()

In [1548]:
sln = solver.solve(problem, b.weights)
sum(oracle.observe(problem, sln, noisy=False))

10

### wait, we can solve problems without knwing the parametrics ?! what about other problems like shortest path 

### Extend to a team of phone marekters

In [1549]:
agents_workload = ("Alex", "Jennifer", "Andrew", "DeAnna", "Jesse")

clients = (
    "Trista", "Meredith", "Aaron", "Bob", "Jillian",
    "Ali", "Ashley", "Emily", "Desiree", "Byron")

In [1550]:
import numpy as np
import sklearn

def score(agent, client):
    try:
        s = 1 / (1 + math.exp(-np.dot(agents_v[agent], clients_v[client])))
    except:
        print (np.dot(agents_v[agent], clients_v[client]))
        return random.random()
    return s
    
def generate_matching_problem(agents, clients):
    num_samples = len(agents) + len(clients)
    samples = sklearn.datasets.make_swiss_roll(num_samples, noise=3, random_state=0)[0]
    random.shuffle(samples)
    clients_v = {clients[i]: samples[i] for i in range(len(clients))}
    agents_v = {agents[i]: samples[i] for i in range(len(agents))}
    match_scores = dict(
        ((agent, client), score(agent, client))
        for agent in agents_workloads
        for client in clients)

    client_time = {client: random.randint(1, 4) for client in clients}
    agents_workload = {agent: random.randint(2, 5) for agent in agents}
    
    return match_scores, client_time, agents_workload, clients_v, agents_v

In [1551]:
def solve_matching(match_scores, client_time, agents_workload):
    agents = agents_workloads.keys()
    clients = client_time.keys()
    
    model = pe.ConcreteModel()
    model.agents = agents_workloads.keys()
    model.clients = clients
    model.match_scores = match_scores
    model.agents_workload = agents_workload

    model.assignments = pe.Var(match_scores.keys(), domain=pe.Binary)
    model.objective = pe.Objective(
            expr=pe.summation(model.match_scores, model.assignments),
            sense=pe.maximize)

    @model.Constraint(model.agents)
    def respect_workload(model, agent):
        return sum(model.assignments[agent, client] * client_time[client] for client in model.clients) <= model.agents_workload[agent]

    @model.Constraint(model.clients)
    def one_agent_per_client(model, client):
        return sum(model.assignments[agent, client] for agent in model.agents) <= 1


    solver = pe.SolverFactory("glpk")
    solver.solve(model)
    sln = [k for k, v in model.assignments.get_values().items() if v == 1.0]
    return sln, sum(match_scores[i] for i in sln)
    

In [1552]:
def solve_matching_greedy(match_scores, client_time, agents_workload):
    agents = agents_workloads.keys()
    clients = client_time.keys()
    matching = sorted(match_scores.items(), key=lambda x: -x[1])
    clients_indicator = {client: 0 for client in clients}
    agents_workload_ = agents_workload.copy()
    sln = []
    for (agent, client), score in matching:
        if clients_indicator[client] == 0 and agents_workload_[agent] >= client_time[client]:
            clients_indicator[client] = 1
            agents_workload_[agent] -= client_time[client]
            sln += [(agent, client)]
    return sln, sum([match_scores[i] for i in sln])



In [1553]:
match_scores, client_time, agents_workload, clients_v, agents_v = generate_matching_problem(agents, clients)

In [1554]:
solve_matching(match_scores, client_time, agents_workload)

([('DeAnna', 'Meredith'),
  ('Andrew', 'Ashley'),
  ('Jesse', 'Jillian'),
  ('DeAnna', 'Emily'),
  ('Jesse', 'Byron'),
  ('Jennifer', 'Bob'),
  ('Alex', 'Trista')],
 7.0)

In [1555]:
solve_matching_greedy(match_scores, client_time, agents_workload)

([('Alex', 'Trista'),
  ('Jennifer', 'Bob'),
  ('Andrew', 'Meredith'),
  ('DeAnna', 'Aaron'),
  ('DeAnna', 'Jillian'),
  ('Jesse', 'Ashley')],
 6.0)

In [1556]:
# the matching optimization problem
class MatchingProblem(Problem):
    def __init__(self, match_scores, client_time, agents_workload, agents_v, clients_v):
        self.match_scores = match_scores
        self.agents_workload = agents_workload
        self.client_time = client_time
        self.features = {(agent, client): np.hstack([agents_v[agent], clients_v[client]]) 
                         for agent, client in match_scores.keys()}


class MatchingProblemModel(ProblemModel):
    def __init__(self, match_scores, weights_known=False):
        if weights_known:
            self.weights = match_scores.copy()
        else:
            self.weights = {i: random.random() for i in match_scores.keys()}
        self.arms = self.weights.keys()  
        
    def get_all_weights(self):
        return self.weights
    
    def set_weights(self, weights):
        for k, v in weights.items():
            self.weights[k] = v
    

class MatchingSolver(Solver):
    def __init__(self):
        pass 
    
    def solve(self, problem, weights):
        return solve_matching(weights, problem.client_time, problem.agents_workload)
        
        
class MatchingOracle(Oracle):
    def __init__(self, match_scores, noise_factor=3.0):
        self.weights = match_scores.copy()
        self.arms = self.weights.keys()  
        self.noise_factor = noise_factor
        
    def get_weight(self, x, noisy=False):
        if noisy:
            return self.weights[x] + (random.random() - 0.5) * self.noise_factor
        else:
            return self.weights[x]
        
    def observe(self, problem, solution, noisy=True):
        return [self.get_weight(x, noisy=noisy) for x in solution]
        
        

In [1590]:
# CombTS
class CombLinTs:
    def __init__(self, problem, p_lambda, p_sigma, solver, oracle):
        self.d = np.array(list(problem.features.values())).shape[1]
        self.p_lambda = p_lambda
        self.p_sigma = p_sigma
        self.sigma = (p_lambda ** 2) * np.eye(self.d)
        self.theta = np.zeros(self.d)
        self.solver = solver
        self.oracle = oracle
        
    def sample_theta(self):
        return np.random.multivariate_normal(self.theta, self.sigma)
    
    def update_params(self, wAt):
        theta = self.theta
        sigma = self.sigma
        
        n = len(wAt)
        
        for k, v in wAt.items():
            f_vec = np.expand_dims(problem.features[k], axis=1)
            t1 = np.matmul(sigma, np.matmul(f_vec, f_vec.T))
            t2 = np.matmul(f_vec.T, np.matmul(sigma, f_vec)) + self.p_sigma ** 2
            t3 = np.matmul(sigma, f_vec)
            t4 = np.matmul(np.matmul(f_vec.T, sigma), f_vec) + self.p_sigma ** 2
            theta = np.matmul((np.eye(sigma.shape[0]) - t1 / t2), theta) + \
                        np.squeeze(t3 / t4) * wAt[k]

            t1 = np.matmul(np.matmul(sigma, np.matmul(f_vec, f_vec.T)), sigma)
            t2 = np.matmul(np.matmul(f_vec.T, sigma), f_vec) + self.p_sigma ** 2
            sigma = sigma - t1/t2
    
        self.theta = theta
        self.sigma = sigma
            
    def bandit_iter(self, problem):
        theta = self.sample_theta()
        At, _ = self.solver.solve(problem, {k: np.dot(v, theta) for k, v in problem.features.items()})
        wAt = self.oracle.observe(problem, At, noisy=True)
        wAt = dict(zip(At, wAt))
        self.update_params(wAt)
        
    def solve(self, problem):
        theta = self.sample_theta()
        sln, _ = self.solver.solve(problem, {k: np.dot(v, theta) for k, v in problem.features.items()})
        return sln

In [1634]:
match_scores, client_time, agents_workload, clients_v, agents_v = generate_matching_problem(agents, clients)
problem = MatchingProblem(match_scores, client_time, agents_workload, agents_v, clients_v)
problem_model = MatchingProblemModel(match_scores, weights_known=False)
oracle = MatchingOracle(match_scores, noise_factor=2)
solver = MatchingSolver()
sln, w = solver.solve(problem, problem_model.get_all_weights())
np.sum(oracle.observe(problem, At, noisy=False))


6.00008340099822

In [1635]:
problem_model = MatchingProblemModel(match_scores, weights_known=True)
sln, w = solver.solve(problem, problem_model.get_all_weights())
np.sum(oracle.observe(problem, sln, noisy=False))


8.0

In [1638]:
p_lambda = 10.
p_sigma = 0.1
features_dim =  10
bandit = CombLinTs(p_lambda=p_lambda,
                     p_sigma=p_sigma, problem=problem, solver=solver, oracle=oracle)

for i in range(100):
    bandit.bandit_iter(problem=problem)
    if i % 10 == 0:
        At = bandit.solve(problem)
        print (np.sum(oracle.observe(problem, At, noisy=False)))

7.0
7.0
8.0
8.0
7.0
7.0
8.0
8.0
8.0
8.0


* scale up all pairs 
* We assumed all weight are random, what if they are normal with different means and variances
* Emperical bayes for new joiners 
* Extend to team
* spending more time is useful
* Streamline with Tensorflow probability

In [1352]:
# TODO: the bandit example
# TODO: drwa gunt chart
# TODO: shortest path 
# TODO: Knapsack

6

You may want to do both selection and schedualing, learn improtrance of each client, start from reasonable wights for faster exporation or add more cool features. Very well, we will go through after we introduce some cool tools. For the time being, lets sharpen our skills by solving 2 other problems.

In [1535]:
# shortest path 

In [1536]:
#knapsac packing

**The dauntless conclusion**: *Be a modeling super star, dont fear missing info!*